In [1]:
import torch, time, pickle
import torch_geometric as tg
from torch_geometric.data import Data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import os.path as osp
import networkx as nx
path='../../../../../../tigress/cj1223/merger_trees/isotrees/'
def convert(d,p):
    dfin=[]
    if len(p)!=len(np.unique(p)):
        print('Wrong order of prog/desc')
    else:
        no=d[0]
        for desc in d:
            if desc==no:
                dfin.append(0)
            else:
                dfin.append(p.index(desc)+1)
    return dfin, np.arange(1, 1+len(p))

In [2]:
# cols=np.array([0,2,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,35]+list(range(37,60)))
is_cat=[0,1,0,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,\
        0,0,0,0,0,0,0,0,0,0,0]
cols=np.array([0,2,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,35]+list(range(37,60)))
# cols=np.array([0,2,4,5,6,7,8,10])
case='222_all'
is_cat=np.array([bool(i) for i in is_cat])
# scaler = pickle.load(open('transformer0.pkl', 'rb'))

In [3]:
pd1=pd.read_table(path+f'isotree_0_0_0.dat', skiprows=0, delimiter='\s+') # consider enforcing dtypes upon import
raw=pd1.drop(axis=0, index=np.arange(50)).reset_index()
trees=raw[raw.isna()['A[z]']] 
halos=raw[~raw.isna()['A[z]']] 
f,it="float64", "int64"
cs=[f,it,f,it,it,it,it,it,it,f,f,f,f,f,it,f,f,f,f,f,f,f,f,f,f,f]
dicts = {}
keys = halos.columns[1:26]
castto = cs
for d, key in enumerate(keys):
        dicts[key] = castto[d]
halos=halos.astype(dicts)

/tmp/ipykernel_32423/1985251666.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  pd1=pd.read_table(path+f'isotree_0_0_0.dat', skiprows=0, delimiter='\s+') # consider enforcing dtypes upon import


In [4]:
hard=[0,2,15]
def logit(x):
    return np.log10((x+0.001)/(1.01-x))
##logit transform the hards
for h in hard:
    halos[halos.columns[h+1]]=logit(halos[halos.columns[h+1]])
log=[10,38, 39, 40, 41, 42]

def logt(x):
    return np.log10(x+1)
for l in log:
    halos[halos.columns[l+1]]=logt(halos[halos.columns[l+1]])
tcols=np.array(cols[~is_cat[cols]])+1

In [5]:
spli=np.split(np.array(halos)[:,1:], np.array(trees.iloc[1:].index)-np.arange(1,len(trees.index)))
splits=[]
for s in spli:
    if s[0,10]>10:
        splits.append(s)
splits=np.array(splits, dtype=object)
split=[]
for tree in splits:
    s=tree[np.logical_or(tree[:,3] == -1,tree[:,4]!=1)]
    split.append(s)
split=np.array(split, dtype=object)

In [10]:
halos.columns
# not_include=np.array([0,1,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,\
#         0,0,0,0,0,0,0,0,0,0,0]) 
# is_cat=[0,1,0,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,\
#         0,0,0,0,0,0,0,0,0,0,0]
# len(not_include), len(is_cat)

Index(['index', '#scale(0)', 'id(1)', 'desc_scale(2)', 'desc_id(3)',
       'num_prog(4)', 'pid(5)', 'upid(6)', 'desc_pid(7)', 'phantom(8)',
       'sam_Mvir(9)', 'Mvir(10)', 'Rvir(11)', 'rs(12)', 'vrms(13)', 'mmp?(14)',
       'scale_of_last_MM(15)', 'vmax(16)', 'x(17)', 'y(18)', 'z(19)', 'vx(20)',
       'vy(21)', 'vz(22)', 'Jx(23)', 'Jy(24)', 'Jz(25)', 'Spin(26)',
       'Breadth_first_ID(27)', 'Depth_first_ID(28)', 'Tree_root_ID(29)',
       'Orig_halo_ID(30)', 'Snap_idx(31)',
       'Next_coprogenitor_depthfirst_ID(32)',
       'Last_progenitor_depthfirst_ID(33)', 'Last_mainleaf_depthfirst_ID(34)',
       'Tidal_Force(35)', 'Tidal_ID(36)', 'Rs_Klypin', 'Mvir_all', 'M200b',
       'M200c', 'M500c', 'M2500c', 'Xoff', 'Voff', 'Spin_Bullock', 'b_to_a',
       'c_to_a', 'A[x]', 'A[y]', 'A[z]', 'b_to_a(500c)', 'c_to_a(500c)',
       'A[x](500c)', 'A[y](500c)', 'A[z](500c)', 'T/|U|', 'M_pe_Behroozi',
       'M_pe_Diemer', 'Halfmass_Radius'],
      dtype='object')

In [6]:
samp='~/../../../tigress/cj1223/merger_trees/samout/'
ex=f'0_0_0/'
ex+='galprop_0-99.dat'
pdc=pd.read_table(samp+ex, skiprows=0, delimiter=',', nrows=41, header=None)
newcols=pdc.iloc[:,0]
pds=pd.read_table(samp+ex, skiprows=41, delimiter='\s+', header=None)
pds.columns=np.array(newcols)

In [7]:
np.array(newcols)

array(['# 0 halo_index (long) ', '# 1 birthhaloid (long long)',
       '# 2 roothaloid (long long)', '# 3 redshift',
       '# 4 sat_type 0= central',
       '# 5 mhalo total halo mass [1.0E09 Msun]',
       '# 6 m_strip stripped mass [1.0E09 Msun]',
       '# 7 rhalo halo virial radius [Mpc)]',
       '# 8 mstar stellar mass [1.0E09 Msun]',
       '# 9 mbulge stellar mass of bulge [1.0E09 Msun] ',
       '# 10 mstar_merge stars entering via mergers] [1.0E09 Msun]',
       '# 11 v_disk rotation velocity of disk [km/s] ',
       '# 12 sigma_bulge velocity dispersion of bulge [km/s]',
       '# 13 r_disk exponential scale radius of stars+gas disk [kpc] ',
       '# 14 r_bulge 3D effective radius of bulge [kpc]',
       '# 15 mcold cold gas mass in disk [1.0E09 Msun]',
       '# 16 mHI cold gas mass [1.0E09 Msun]',
       '# 17 mH2 cold gas mass [1.0E09 Msun]',
       '# 18 mHII cold gas mass [1.0E09 Msun]',
       '# 19 Metal_star metal mass in stars [Zsun*Msun]',
       '# 20 Metal_cold

In [16]:
pdcen=pds[pds[pds.columns[4]]==0]
pdcen0=pdcen[pdcen[pdcen.columns[3]]==0.0]
pdcen0[pdcen0.columns[1]]==pdcen0[pdcen0.columns[2]]

74         True
155        True
252        True
317        True
387        True
           ... 
1060248    True
1060309    True
1060370    True
1060418    True
1060433    True
Length: 9513, dtype: bool

In [ ]:
# dat=[]
# for i in range(i0,i1+1):
#     for j in range(j0,j1+1):
#         for k in range(k0,k1+1):
#             print(f'Loading isotree {i}_{j}_{k}')
#             start=time.time()
            pd1=pd.read_table(path+f'isotree_{i}_{j}_{k}.dat', skiprows=0, delimiter='\s+') # consider enforcing dtypes upon import
            print(f'isotree {i}_{j}_{k} loaded, restructuring')
            raw=pd1.drop(axis=0, index=np.arange(50)).reset_index()
            trees=raw[raw.isna()['A[z]']] 
            halos=raw[~raw.isna()['A[z]']] 
            f,it="float64", "int64"
            cs=[f,it,f,it,it,it,it,it,it,f,f,f,f,f,it,f,f,f,f,f,f,f,f,f,f,f]
            dicts = {}
            keys = halos.columns[1:26]
            castto = cs
            for d, key in enumerate(keys):
                    dicts[key] = castto[d]
            halos=halos.astype(dicts)
            
            hard=[0,2,15]
            def logit(x):
                return np.log10((x+0.001)/(1.01-x))
            ##logit transform the hards
            for h in hard:
                halos[halos.columns[h+1]]=logit(halos[halos.columns[h+1]])
            log=[10,38, 39, 40, 41, 42]

            def logt(x):
                return np.log10(x+1)
            for l in log:
                halos[halos.columns[l+1]]=logt(halos[halos.columns[l+1]])
            tcols=np.array(cols[~is_cat[cols]])+1
#             halos[halos.columns[tcols]]=scaler.transform(halos[halos.columns[tcols]])
            
            print(f'Splitting to tree')
            spli=np.split(np.array(halos)[:,1:], np.array(trees.iloc[1:].index)-np.arange(1,len(trees.index)))
            splits=[]
            for s in spli:
                if s[0,10]>10:
                    splits.append(s)
            splits=np.array(splits, dtype=object)
            split=[]
            for tree in splits:
                s=tree[np.logical_or(tree[:,3] == -1,tree[:,4]!=1)]
                split.append(s)
            split=np.array(split, dtype=object)
            print('Split done')
            print('Loading targets')
            samp='~/../../../tigress/mcranmer/merger_trees/samout/'
            ex=f'{i}_{j}_{k}/'
            ex+='galprop_0-99.dat'
            pdc=pd.read_table(samp+ex, skiprows=0, delimiter=',', nrows=41, header=None)
            newcols=pdc.iloc[:,0]
            pds=pd.read_table(samp+ex, skiprows=41, delimiter='\s+', header=None)
            pds.columns=np.array(newcols)
            pd0=pds[pds[pds.columns[3]]==0.00] # subhaloes
            pdcen=pd0[(pd0[pds.columns[1]]==pd0[pds.columns[2]])] ##central haloes
            
            rhalid=np.array(pdcen[pds.columns[2]])
            halwgal=[]
            ids=[]
            out=[]
            for idx, tree in enumerate(split):
                if tree[0,1] in rhalid:
                    halwgal.append(tree)
                    ids.append(idx)
                    index=np.where(rhalid==tree[0,1])
                    out.append(np.array(pdcen.iloc[index])[0][8]) ## target variable
            stop=time.time()
            hraw=np.array(splits[ids], dtype=object)
            hals=[]
            pr,de=[],[]
            print('Making merger tree')
            for n in tqdm(range(len(halwgal))):
                h=halwgal[n]
                roots=h[h[:,4]==0]
            #     mergers=h[np.logical_and(h[:,4]>1,h[:,3]!=-1)] #remove self-loop, is broken since some roots go directly to final
                mergers=h[h[:,4]>1]
                final=h[h[:,3]==-1]
                pro, des=[],[]
                for r in roots:
                    descid=hraw[n][:,3][np.where(r[1]==hraw[n][:,1])]
                    while descid not in mergers[:,1]:
                        descid=hraw[n][:,3][np.where(descid==hraw[n][:,1])] #consider adding the number of steps it went through
                    pro.append(r[1])
                    des.append(descid[0])
                m=mergers[mergers[:, 0].argsort()] #sort by redshift, starting at early times
                mdes,mpro=list(m[:,1][1:]), list(m[:,1])
                des+=mdes
                pro+=mpro
                des.append(final[:,1][0])
                des,pro=convert(des[::-1],pro[::-1])
                hal2=np.vstack([final,np.flip(m,axis=0).copy(),np.flip(roots,axis=0).copy()])
                hal2=hal2[:,cols] 
                hals.append(hal2)
                pr.append([int(p) for p in pro])
                de.append([int(d) for d in des])
            hals=np.array(hals,dtype=object)
            out=np.log(out)/5 #homemade scaling
            for n in tqdm(range(len(out))):
                edge_index = torch.tensor([pr[n],de[n]], dtype=torch.long)
                x = torch.tensor(hals[n], dtype=torch.float)

                y=torch.tensor(out[n], dtype=torch.float)
                graph=Data(x=x, edge_index=edge_index, y=y)
                dat.append(graph)
            stop=time.time()
            print(f'Done with tree. \n Time elapsed {stop-start} s')
print("Saving dataset")
if not osp.exists(f'../../../../scratch/gpfs/cj1223/GraphStorage/{case}'):
    os.mkdir(f'../../../../scratch/gpfs/cj1223/GraphStorage/{case}')
                  
with open(f'../../../../scratch/gpfs/cj1223/GraphStorage/{case}/data.pkl', 'wb') as handle:
    pickle.dump(dat, handle)

Loading isotree 0_0_2


/home/cj1223/.conda/envs/juptorch/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


isotree 0_0_2 loaded, restructuring


In [ ]:
print(f'Loading isotree {i}_{j}_{k}')
start=time.time()
pd1=pd.read_table(path+f'isotree_{i}_{j}_{k}.dat', skiprows=0, delimiter='\s+') # consider enforcing dtypes upon import
print(f'isotree {i}_{j}_{k} loaded, restructuring')
raw=pd1.drop(axis=0, index=np.arange(50)).reset_index()
trees=raw[raw.isna()['A[z]']] 
halos=raw[~raw.isna()['A[z]']] 
f,it="float64", "int64"
cs=[f,it,f,it,it,it,it,it,it,f,f,f,f,f,it,f,f,f,f,f,f,f,f,f,f,f]
dicts = {}
keys = halos.columns[1:26]
castto = cs
for d, key in enumerate(keys):
        dicts[key] = castto[d]
halos=halos.astype(dicts)

hard=[0,2,15]
def logit(x):
    return np.log10((x+0.001)/(1.01-x))
##logit transform the hards
for h in hard:
    halos[halos.columns[h+1]]=logit(halos[halos.columns[h+1]])
log=[10,38, 39, 40, 41, 42]

def logt(x):
    return np.log10(x+1)
for l in log:
    halos[halos.columns[l+1]]=logt(halos[halos.columns[l+1]])
tcols=np.array(cols[~is_cat[cols]])+1
halos[halos.columns[tcols]]=scaler.transform(halos[halos.columns[tcols]])



In [ ]:
print(f'Restructured, splitting to tree')
spli=np.split(np.array(halos)[:,1:], np.array(trees.iloc[1:].index)-np.arange(1,len(trees.index)))
splits=[]
for s in spli:
    if s[0,10]>1:
        splits.append(s)
splits=np.array(splits, dtype=object)
split=[]
for tree in splits:
    s=tree[np.logical_or(tree[:,3] == -1,tree[:,4]!=1)]
    split.append(s)
split=np.array(split, dtype=object)
print('Split done')
print(len(split))
print('Loading targets')

In [ ]:
samp='~/../../../tigress/mcranmer/merger_trees/samout/'
ex=f'{0}_{j}_{k}/'
ex+='galprop_0-99.dat'
pdc=pd.read_table(samp+ex, skiprows=0, delimiter=',', nrows=41, header=None)
newcols=pdc.iloc[:,0]
pds=pd.read_table(samp+ex, skiprows=41, delimiter='\s+', header=None)
pds.columns=np.array(newcols)
pd0=pds[pds[pds.columns[3]]==0.00] # subhaloes
pdcen=pd0[(pd0[pds.columns[1]]==pd0[pds.columns[2]])] ##central haloes

rhalid=np.array(pdcen[pds.columns[2]])
halwgal=[]
ids=[]
out=[]
for idx, tree in enumerate(split):
    if tree[0,1] in rhalid:
        halwgal.append(tree)
        ids.append(idx)
        index=np.where(rhalid==tree[0,1])
        out.append(np.array(pdcen.iloc[index])[0][8]) ## target variable
stop=time.time()
hraw=np.array(splits[ids], dtype=object)
len(hraw)

In [ ]:
hals=[]
pr,de=[],[]
print('Making merger tree')
for i in tqdm(range(len(halwgal))):
    h=halwgal[i]
    roots=h[h[:,4]==0]
#     mergers=h[np.logical_and(h[:,4]>1,h[:,3]!=-1)] #remove self-loop, is broken since some roots go directly to final
    mergers=h[h[:,4]>1]
    final=h[h[:,3]==-1]
    pro, des=[],[]
    for r in roots:
        descid=hraw[i][:,3][np.where(r[1]==hraw[i][:,1])]
        while descid not in mergers[:,1]:
            descid=hraw[i][:,3][np.where(descid==hraw[i][:,1])] #consider adding the number of steps it went through
        pro.append(r[1])
        des.append(descid[0])
    m=mergers[mergers[:, 0].argsort()] #sort by redshift, starting at early times
    mdes,mpro=list(m[:,1][1:]), list(m[:,1])
    des+=mdes
    pro+=mpro
    des.append(final[:,1][0])
    des,pro=convert(des[::-1],pro[::-1])
    hal2=np.vstack([final,np.flip(m,axis=0).copy(),np.flip(roots,axis=0).copy()])
    hal2=hal2[:,cols] 
    hals.append(hal2)
    pr.append([int(p) for p in pro])
    de.append([int(d) for d in des])
hals=np.array(hals,dtype=object)
for n in tqdm(range(len(out))):
    edge_index = torch.tensor([pr[n],de[n]], dtype=torch.long)
    x = torch.tensor(hals[n], dtype=torch.float)

    y=torch.tensor(out[n], dtype=torch.float)
    graph=Data(x=x, edge_index=edge_index, y=y)
    dat.append(graph)
stop=time.time()
print(f'Done with tree. \n Time elapsed {stop-start} s')

In [19]:
meta = pickle.load(open(osp.expanduser(f'~/../../../scratch/gpfs/cj1223/GraphStorage/test_all_8t_z0.0_None/data.pkl'), 'rb'))

In [22]:
meta[0].x[0]

tensor([ 1.0000e+00,  0.0000e+00,  1.0000e+00,  1.0205e+01,  5.1285e+01,
         5.3800e-01,  5.5700e+01,  1.0000e+00,  1.1096e-01,  6.2700e+01,
         7.5915e-07, -3.0353e-06, -3.4487e-05,  1.2167e+00,  1.4476e+00,
         1.0222e+01,  1.0218e+01,  1.0182e+01,  1.0146e+01,  1.0066e+01,
         6.1221e-01,  7.0000e-01,  2.2090e-02,  9.6428e-01,  8.2845e-01,
         2.6537e+00, -1.7162e+00,  3.0579e-01,  9.3909e-01,  8.1173e-01,
        -1.8847e+00,  2.2702e+00,  1.9854e-01,  5.1880e-01,  1.0568e+01,
         9.9493e+00,  8.9350e+00])

In [23]:
big = pickle.load(open(osp.expanduser(f'~/../../../scratch/gpfs/cj1223/GraphStorage/vlarge_all_4t_z0.0_None/data.pkl'), 'rb'))

In [26]:
big[0].x[0,:10],big[0].x[0,16:]

(tensor([ 1.0000,  0.0000,  1.0000, 10.2055, 51.2850,  0.5380, 55.7000,  1.0000,
          0.1110, 62.7000]),
 tensor([ 7.5915e-07, -3.0353e-06, -3.4487e-05,  1.2167e+00,  1.4476e+00,
          1.0222e+01,  1.0218e+01,  1.0182e+01,  1.0146e+01,  1.0066e+01,
          6.1221e-01,  7.0000e-01,  2.2090e-02,  9.6428e-01,  8.2845e-01,
          2.6537e+00, -1.7162e+00,  3.0579e-01,  9.3909e-01,  8.1173e-01,
         -1.8847e+00,  2.2702e+00,  1.9854e-01,  5.1880e-01,  1.0568e+01,
          9.9493e+00,  8.9350e+00]))

In [12]:
pds[pds.columns[0:9]].describe()

,# 0 halo_index (long),# 1 birthhaloid (long long),# 2 roothaloid (long long),# 3 redshift,# 4 sat_type 0= central,# 5 mhalo total halo mass [1.0E09 Msun],# 6 m_strip stripped mass [1.0E09 Msun],# 7 rhalo halo virial radius [Mpc)],# 8 mstar stellar mass [1.0E09 Msun]
count,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06
mean,1.416755e+06,6.270766e+08,1.123081e+09,1.304976e+00,2.168065e-01,9.834023e+00,9.121821e+00,2.429380e-02,9.874990e-02
std,7.454591e+05,2.934791e+08,9.856745e+04,1.543315e+00,4.964236e-01,1.503040e+02,1.484889e+02,1.837911e-02,2.874288e+00
min,1.700000e+01,1.088964e+06,1.122969e+09,0.000000e+00,0.000000e+00,8.850330e-01,6.395800e-02,2.147070e-03,0.000000e+00
25%,8.300290e+05,3.894396e+08,1.123000e+09,2.977240e-01,0.000000e+00,1.213460e+00,1.059940e+00,1.406250e-02,1.056172e-05
50%,1.401494e+06,6.377958e+08,1.123025e+09,7.574380e-01,0.000000e+00,1.888470e+00,1.662240e+00,2.125380e-02,1.176925e-04
75%,2.092867e+06,8.734663e+08,1.123194e+09,1.666670e+00,0.000000e+00,3.959260e+00,3.525240e+00,2.881460e-02,7.063025e-04
max,2.652324e+06,1.123278e+09,1.123278e+09,1.498980e+01,5.000000e+00,1.927960e+04,1.927960e+04,7.064030e-01,3.347790e+02


In [13]:
pds[pds.columns[9:18]].describe()

,# 9 mbulge stellar mass of bulge [1.0E09 Msun],# 10 mstar_merge stars entering via mergers] [1.0E09 Msun],# 11 v_disk rotation velocity of disk [km/s],# 12 sigma_bulge velocity dispersion of bulge [km/s],# 13 r_disk exponential scale radius of stars+gas disk [kpc],# 14 r_bulge 3D effective radius of bulge [kpc],# 15 mcold cold gas mass in disk [1.0E09 Msun],# 16 mHI cold gas mass [1.0E09 Msun],# 17 mH2 cold gas mass [1.0E09 Msun]
count,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06
mean,3.366072e-02,1.563831e-02,2.645968e+01,3.932111e+02,6.049376e-01,1.387612e-02,1.468070e-01,5.869764e-02,1.266740e-02
std,2.086846e+00,1.420681e+00,1.507525e+01,5.931494e+03,5.980792e-01,1.659667e-01,9.015949e-01,3.230130e-01,2.386238e-01
min,0.000000e+00,0.000000e+00,1.117000e+01,3.301110e+00,2.029300e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,1.741910e+01,1.169210e+01,2.895230e-01,0.000000e+00,5.901878e-03,8.991905e-04,4.383375e-07
50%,2.295275e-06,0.000000e+00,2.276780e+01,5.033810e+01,4.714215e-01,6.949960e-08,2.259300e-02,8.535270e-03,1.282290e-05
75%,6.520337e-05,6.733410e-06,3.052470e+01,1.836400e+02,7.463448e-01,3.241990e-05,6.988375e-02,3.068907e-02,2.328847e-04
max,3.124400e+02,2.212770e+02,4.088650e+02,4.159280e+05,4.305380e+01,1.399260e+01,4.120680e+01,1.700110e+01,1.853730e+01


In [14]:
pds[pds.columns[18:27]].describe()

,# 18 mHII cold gas mass [1.0E09 Msun],# 19 Metal_star metal mass in stars [Zsun*Msun],# 20 Metal_cold metal mass in cold gas [Zsun*Msun],# 21 sfr instantaneous SFR [Msun/yr],# 22 sfrave20myr SFR averaged over 20 Myr [Msun/yr],# 23 sfrave100myr SFR averaged over 100 Myr [Msun/yr],# 24 sfrave1gyr SFR averaged over 1 Gyr [Msun/yr],# 25 mass_outflow_rate [Msun/yr],# 26 metal_outflow_rate [Msun/yr]
count,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06
mean,3.719329e-02,1.262311e-01,2.513469e-02,2.417977e-02,2.421254e-02,2.385629e-02,2.255303e-02,2.185317e-01,4.760611e-02
std,1.993134e-01,5.545877e+00,5.601971e-01,7.784900e-01,7.749709e-01,7.403808e-01,6.707843e-01,1.743012e+00,1.437252e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.721805e-03,7.892567e-09,4.591687e-06,3.104203e-07,3.353165e-07,3.303133e-07,2.649598e-07,6.840350e-04,4.691862e-07
50%,7.367645e-03,2.750555e-07,3.333900e-05,8.116635e-06,9.001880e-06,9.031680e-06,8.565745e-06,9.482565e-03,1.359850e-05
75%,2.036657e-02,3.763478e-06,2.013153e-04,1.446473e-04,1.588945e-04,1.599000e-04,1.500550e-04,7.757402e-02,2.634878e-04
max,1.199640e+01,7.106480e+02,5.094510e+01,2.549480e+02,2.444550e+02,1.216880e+02,1.010150e+02,3.540410e+02,3.010690e+02


In [15]:
pds[pds.columns[27:36]].describe()

,# 27 mBH black hole mass [1.0E09 Msun],# 28 maccdot accretion rate onto BH [Msun/yr],# 29 maccdot_radio accretion rate in radio mode [Msun/yr],# 30 tmerge time since last merger [Gyr],# 31 tmajmerge time since last major merger [Gyr],# 32 mu_merge mass ratio of last merger [],# 33 t_sat time since galaxy became a satellite in this halo [Gyr],# 34 r_fric distance from halo center [Mpc],# 35 x_position x coordinate [cMpc]
count,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06
mean,2.377830e-04,2.265473e-05,7.443969e-06,4.920846e+01,8.229237e+01,1.312852e-01,-8.046898e+01,1.884827e-02,1.539828e+01
std,1.754207e-02,1.058063e-02,1.002243e-03,5.096084e+01,4.379863e+01,1.532992e-01,3.914559e+01,6.521342e-02,1.967860e+01
min,1.000000e-05,0.000000e+00,0.000000e+00,-2.357800e-01,-2.357800e-01,0.000000e+00,-9.900000e+01,0.000000e+00,-6.756220e-02
25%,1.000500e-05,0.000000e+00,0.000000e+00,2.760960e+00,1.006320e+02,0.000000e+00,-9.900000e+01,0.000000e+00,7.833270e+00
50%,2.000540e-05,0.000000e+00,7.105715e-10,8.407740e+00,1.045560e+02,1.157730e-01,-9.900000e+01,0.000000e+00,1.252510e+01
75%,3.001890e-05,0.000000e+00,3.116150e-09,1.048800e+02,1.089620e+02,2.104760e-01,-9.900000e+01,0.000000e+00,1.603200e+01
max,3.092650e+00,1.007740e+01,1.708400e-01,1.137020e+02,1.137020e+02,9.988070e-01,1.243640e+01,7.349710e-01,1.109060e+02


In [16]:
pds[pds.columns[36:41]].describe()

,# 36 y_position y coordinate [cMpc],# 37 z_position z coordinate [cMpc],# 38 vx x component of velocity [km/s],# 39 vy y component of velocity [km/s],# 40 vz z component of velocity [km/s]
count,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06,1.060434e+06
mean,1.434490e+01,1.465827e+01,2.041009e+02,5.595665e+01,1.304421e+02
std,2.108687e+01,1.837565e+01,1.142881e+02,1.168890e+02,1.077521e+02
min,-7.710760e-01,-2.169100e-01,-9.237020e+02,-9.243540e+02,-9.861050e+02
25%,5.289060e+00,7.585220e+00,1.305400e+02,-2.249000e+01,8.310000e+01
50%,1.046450e+01,1.200885e+01,2.010300e+02,5.336000e+01,1.365800e+02
75%,1.534260e+01,1.619440e+01,2.681800e+02,1.333940e+02,1.939300e+02
max,1.110640e+02,1.110400e+02,1.384570e+03,1.289870e+03,1.132460e+03
